In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [3]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [4]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [5]:
def L2_Reg(x1):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized
    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe UV Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
    y = df['Peak_eV']
    z = df['FWHM_eV']
    group = df['Expt. No']

    regressor_peak = Ridge(alpha=x1)
    regressor_fwhm = Ridge(alpha=x1)
    regressor_cascade = Ridge(alpha=x1)

    gkf = GroupKFold(n_splits=10)
    matrix = np.zeros([10,13])
    matrix_fwhm = np.zeros([10,13])
    matrix_cascade = np.zeros([10,14])
    for i in range(0,1):
        j = 0
        for train_index, test_index in gkf.split(X, groups=group):
            #Separate out training and test sets
            X_train = X.iloc[train_index, :]
            X2_train = X2.iloc[train_index, :]
            y_train = y[train_index]
            z_train = z[train_index]
            X_test = X.iloc[test_index, :]
            y_test = y[test_index]
            z_test = z[test_index]

            #prediction for Peak
            regressor_peak.fit(X_train, y_train)
            y_pred = regressor_peak.predict(X_test)
            #prediction for FWHM (direct)
            regressor_fwhm.fit(X_train, z_train)
            y_pred_fwhm = regressor_fwhm.predict(X_test)
            #prediction for FWHM (cascade)
            regressor_cascade.fit(X2_train, z_train)
            X_test.insert(5, "Peak_eV", regressor_peak.predict(X_test), True)
            y_pred_cascade = regressor_cascade.predict(X_test)

            #errors in eV
            matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
            matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
            matrix[10*i + j,2] = rae(y_test, y_pred)
            matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

            matrix_fwhm[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,1] = metrics.r2_score(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,2] = rae(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))

            matrix_cascade[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,1] = metrics.r2_score(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,2] = rae(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))

            #errors in nm
            #UNnormalize the data first
            y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            z_test = z_test*(0.35)+0.05
            y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
            y_pred_cascade = y_pred_cascade*(0.35)+0.05
            matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
            matrix[10*i + j,8:13] = regressor_peak.coef_

            matrix_fwhm[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
            matrix_fwhm[10*i + j,8:13] = regressor_fwhm.coef_

            matrix_cascade[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
            matrix_cascade[10*i + j,8:14] = regressor_cascade.coef_

            j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = matrix.mean(0)[:8]
    matrix_2[0,8:16] = np.std(matrix[:,:8], axis=0)
    matrix_2[0,16:21] = matrix.mean(0)[8:13]

    matrix_3 = np.zeros([1,21])
    matrix_3[0,:8] = matrix_fwhm.mean(0)[:8]
    matrix_3[0,8:16] = np.std(matrix_fwhm[:,:8], axis=0)
    matrix_3[0,16:21] = matrix_fwhm.mean(0)[8:13]

    matrix_4 = np.zeros([1,22])
    matrix_4[0,:8] = matrix_cascade.mean(0)[:8]
    matrix_4[0,8:16] = np.std(matrix_cascade[:,:8], axis=0)
    matrix_4[0,16:22] = matrix_cascade.mean(0)[8:14]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0], matrix_2[0,8])
    print("R2 (norm, eV):", matrix_2[0,1], matrix_2[0,9])
    print("RAE (norm, eV):", matrix_2[0,2], matrix_2[0,10])
    print("RMSE (norm, eV):", matrix_2[0,3], matrix_2[0,11])
    print("Importances", matrix_2[0,8:13])
    print("MAE (nm):", matrix_2[0,4], matrix_2[0,12])
    print("R2 (nm):", matrix_2[0,5], matrix_2[0,13])
    print("RAE (nm):", matrix_2[0,6], matrix_2[0,14])
    print("RMSE (nm):", matrix_2[0,7], matrix_2[0,15])

    print('\033[1m' + 'Absorption FWHM (direct)' + '\033[0m')
    print("MAE (norm, eV):", matrix_3[0,0], matrix_3[0,8])
    print("R2 (norm, eV):", matrix_3[0,1], matrix_3[0,9])
    print("RAE (norm, eV):", matrix_3[0,2], matrix_3[0,10])
    print("RMSE (norm, eV):", matrix_3[0,3], matrix_3[0,11])
    print("Importances", matrix_3[0,8:13])
    print("MAE (nm):", matrix_3[0,4], matrix_3[0,12])
    print("R2 (nm):", matrix_3[0,5], matrix_3[0,13])
    print("RAE (nm):", matrix_3[0,6], matrix_3[0,14])
    print("RMSE (nm):", matrix_3[0,7], matrix_3[0,15])

    print('\033[1m' + 'Absorption FWHM (cascade)' + '\033[0m')
    print("MAE (norm, eV):", matrix_4[0,0], matrix_4[0,8])
    print("R2 (norm, eV):", matrix_4[0,1], matrix_4[0,9])
    print("RAE (norm, eV):", matrix_4[0,2], matrix_4[0,10])
    print("RMSE (norm, eV):", matrix_4[0,3], matrix_4[0,11])
    print("Importances", matrix_4[0,8:14])
    print("MAE (nm):", matrix_4[0,4], matrix_4[0,12])
    print("R2 (nm):", matrix_4[0,5], matrix_4[0,13])
    print("RAE (nm):", matrix_4[0,6], matrix_4[0,14])
    print("RMSE (nm):", matrix_4[0,7], matrix_4[0,15])

    return {'loss': matrix_3[0,3], 'status': STATUS_OK }

In [6]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

best = fmin(L2_Reg,
            space=hp.uniform('x1', 0.001, 20),
            algo=tpe.suggest,
            max_evals=100,
            rstate=rstate)

print(best)

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.0945898859482909                                                                                                     
0.016107560714378463                                                                                                   
R2 (norm, eV):                                                                                                         
0.5414494816268409                                                                                                     
0.09568050394175633                                                                                                    
RAE (norm, eV):                                                                                                        
0.6685414666360787                              

[0.00912646 0.21284948 0.10248498 0.01725431 0.73529546 0.24277212]                                                    
MAE (nm):                                                                                                              
3.5998344309763297                                                                                                     
0.7352954647894971                                                                                                     
R2 (nm):                                                                                                               
0.32750812676418606                                                                                                    
0.2427721223374817                                                                                                     
RAE (nm):                                                                                                              
0.7556525373466624                      

MAE (norm, eV):                                                                                                        
0.03573658104837718                                                                                                    
0.009016765591205964                                                                                                   
R2 (norm, eV):                                                                                                         
0.43911542953095195                                                                                                    
0.22363769517661675                                                                                                    
RAE (norm, eV):                                                                                                        
0.6583203885246312                                                                                                     
0.10407692039531102                     

MAE (nm):                                                                                                              
3.5335243409511206                                                                                                     
0.7472199515076327                                                                                                     
R2 (nm):                                                                                                               
0.3268653218756688                                                                                                     
0.26009510204551456                                                                                                    
RAE (nm):                                                                                                              
0.7431170815313588                                                                                                     
0.12633093486199984                     

0.035797376273525                                                                                                      
0.009058200009793906                                                                                                   
R2 (norm, eV):                                                                                                         
0.43822293336816404                                                                                                    
0.22121997934184667                                                                                                    
RAE (norm, eV):                                                                                                        
0.6593306085451616                                                                                                     
0.10391069120855952                                                                                                    
RMSE (norm, eV):                        

...

0.7478665392898333                                                                                                     
R2 (nm):                                                                                                               
0.32142798079884793                                                                                                    
0.26070064866338966                                                                                                    
RAE (nm):                                                                                                              
0.7440552857289658                                                                                                     
0.12648670704900822                                                                                                    
RMSE (nm):                                                                                                             
4.760908535700075                       

R2 (norm, eV):                                                                                                         
0.4383949222721257                                                                                                     
0.22158700493095967                                                                                                    
RAE (norm, eV):                                                                                                        
0.6591508758498646                                                                                                     
0.10393144191711949                                                                                                    
RMSE (norm, eV):                                                                                                       
0.05197076016668758                                                                                                    
0.017630414304853556                    

R2 (nm):                                                                                                               
0.5667162827155681                                                                                                     
0.15210935306671192                                                                                                    
RAE (nm):                                                                                                              
0.6448471640549769                                                                                                     
0.12971049764972953                                                                                                    
RMSE (nm):                                                                                                             
21.817664854207187                                                                                                     
2.6919639956853496                      

0.556571364450445                                                                                                      
0.10599543103822953                                                                                                    
RAE (norm, eV):                                                                                                        
0.6561917538801977                                                                                                     
0.10631392923305931                                                                                                    
RMSE (norm, eV):                                                                                                       
0.11003085838775015                                                                                                    
0.014901799473120845                                                                                                   
Importances                             

0.33295918956494697                                                                                                    
0.25132002534418363                                                                                                    
RAE (nm):                                                                                                              
0.7470122324766801                                                                                                     
0.12216371296632424                                                                                                    
RMSE (nm):                                                                                                             
4.724339783840169                                                                                                      
1.4512079718018953                                                                                                     
100%|███████████████████████████████████